In [1]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda"

In [ ]:
# Load images
# image1 = load_image("https://media.istockphoto.com/id/485371557/photo/twilight-at-spirit-island.jpg?s=612x612&w=0&k=20&c=FSGliJ4EKFP70Yjpzso0HfRR4WwflC6GKfl4F3Hj7fk=")
# image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")
# image1 = Image.open("../../../.vscode/fuji-mountain-in-autumn.jpg")
image1 = load_image("https://artwyrd.com/wp-content/uploads/2021/01/angela-porter-2021-jan-25.jpg")
print(f"Image 1 size: {image1.size}")

# Initialize processor and model
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Can you describe the image?"}
            # {"type": "text", "text": "A real-valued function f defined on the real line is called an even function if f(-t) = f(t) for each real number t. Prove that the set of even functions defined on the real line with the operations of addition and scalar multiplication defined in Example 3 is a vector space."}
        ]
    },
]

FileNotFoundError: [Errno 2] No such file or directory: 'https://artwyrd.com/wp-content/uploads/2021/01/angela-porter-2021-jan-25.jpg'

In [35]:
# Initialize model directly on CUDA without Flash Attention
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16,
    # _attn_implementation="flash_attention_2",  # Commented out Flash Attention
    device_map="cuda",
)
type(model)

transformers.models.idefics3.modeling_idefics3.Idefics3ForConditionalGeneration

In [36]:
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
# inputs = processor(text=prompt, return_tensors="pt")
inputs = inputs.to("cuda")

print(inputs["input_ids"])

# Generate outputs
generated_ids = model.generate(
    **inputs,
    max_new_tokens=500,
    do_sample=False,  # Use greedy decoding (highest logit)
)
generated_texts = processor.batch_decode(
    generated_ids,  # Slice to include only the first token
    skip_special_tokens=True,
)

print(">>", generated_texts[0])

tensor([[    1, 11126,    42,  ...,  9519,  9531,    42]], device='cuda:0')
>> User:<image>Can you describe the image?
Assistant: The image features a scenic view of a mountain, likely Mount Fuji in Japan, with a bridge in the foreground and a body of water in the foreground. The mountain is partially covered with snow, indicating it is likely in the winter season. The sky is clear with a few clouds, and the water in the foreground is calm and still, reflecting the colors of the sky and the mountain. 

The bridge is a long, narrow structure that spans the body of water, connecting the land on both sides. The bridge is made of a light-colored material, possibly wood or concrete, and it appears to be sturdy and well-maintained. The bridge is surrounded by trees, which are mostly green but some have hints of autumn colors, suggesting the season is changing.

The water in the foreground is calm and still, with a few ripples on the surface, reflecting the colors of the sky and the mountain.

In [ ]:
tokenP = [    1, 11126,    42,   330,  1345,    29, 34270,  1517,   275,  4355,
           335,   260,  1345,  1761,   314,  1217,   354,   908,  1517,   585,
           275, 10242,   100,    25,   446,   275,    24,   100,    25,   327,
           971,  1345,  1230,   252,    30,  1053,   307,   338,   260,   932,
           282,   908,  3691,  4355,   335,   260,  1345,  1761,   351,   260,
          4261,   282,  1706,   284, 26727, 17385,  4355,   281, 12066,   216,
            35,   314,   253,  8431,  1898,    30, 49154,   198,  9519,  9531,
            42]
tokenR = [1, 11126, 42, 330, 1345, 29, 34270, 1517, 275, 4355, 335, 260, 1345, 1761, 314, 1217, 354, 908, 1517, 585, 275, 10242, 100, 25, 446, 275, 24, 100, 25, 327, 971, 1345, 1230, 252, 30, 1053, 307, 338, 260, 932, 282, 908, 3691, 4355, 335, 260, 1345, 1761, 351, 260, 4261, 282, 1706, 284, 26727, 17385, 4355, 281, 12066, 216, 35, 314, 253, 8431, 1898, 30, 49154, 198, 9519, 9531, 42]

print("Token same?", tokenP == tokenR)

True


In [37]:
print(processor.tokenizer.chat_template)
"""<|im_start|>{% for message in messages %}{{message['role'] | capitalize}}{% if message['content'][0]['type'] == 'image' %}{{':'}}{% else %}{{': '}}{% endif %}{% for line in message['content'] %}{% if line['type'] == 'text' %}{{line['text']}}{% elif line['type'] == 'image' %}{{ '<image>' }}{% endif %}{% endfor %}<end_of_utterance>{% endfor %}{% if add_generation_prompt %}{{ 'Assistant:' }}{% endif %}"""

"""
<|im_start|>
{% for message in messages %}
    {{message['role'] | capitalize}}
    {% if message['content'][0]['type'] == 'image' %}
        {{':'}}
    {% else %}
        {{': '}}
    {% endif %}
    {% for line in message['content'] %}
        {% if line['type'] == 'text' %}
            {{line['text']}}
        {% elif line['type'] == 'image' %}
            {{ '<image>' }}
        {% endif %}
    {% endfor %}
    <end_of_utterance>]
{% endfor %}
{% if add_generation_prompt %}
    {{ 'Assistant:' }}
{% endif %}
"""

<|im_start|>{% for message in messages %}{{message['role'] | capitalize}}{% if message['content'][0]['type'] == 'image' %}{{':'}}{% else %}{{': '}}{% endif %}{% for line in message['content'] %}{% if line['type'] == 'text' %}{{line['text']}}{% elif line['type'] == 'image' %}{{ '<image>' }}{% endif %}{% endfor %}<end_of_utterance>
{% endfor %}{% if add_generation_prompt %}{{ 'Assistant:' }}{% endif %}


In [12]:
# 🔍 COMPREHENSIVE MODEL VERIFICATION
print("=" * 80)
print("🔍 MODEL ARCHITECTURE VERIFICATION")
print("=" * 80)

# Check model ID and name
print(f"\n📋 Model Information:")
print(f"  Model ID: HuggingFaceTB/SmolVLM-Instruct")
print(f"  Model Class: {type(model).__name__}")
print(f"  Model Module: {type(model).__module__}")
print(f"  Processor Class: {type(processor).__name__}")

# Check model config
print(f"\n⚙️  Model Configuration:")
config = model.config
print(f"  Model Type: {getattr(config, 'model_type', 'Unknown')}")
print(f"  Architecture: {getattr(config, 'architectures', 'Unknown')}")
print(f"  Vision Config: {type(getattr(config, 'vision_config', None)).__name__ if hasattr(config, 'vision_config') else 'None'}")
print(f"  Text Config: {type(getattr(config, 'text_config', None)).__name__ if hasattr(config, 'text_config') else 'None'}")

# Check if it's really SmolVLM or Idefics3
print(f"\n🤔 Is this really SmolVLM?")
print(f"  Config says model_type: {getattr(config, 'model_type', 'Unknown')}")
print(f"  Class name suggests: {'Idefics3' if 'idefics3' in type(model).__name__.lower() else 'Other'}")

# Check model parameters
print(f"\n📊 Model Size:")
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Model size: ~{total_params / 1e9:.1f}B parameters")

# Check config details
print(f"\n🔧 Key Config Details:")
if hasattr(config, 'text_config'):
    text_config = config.text_config
    print(f"  Text model: {getattr(text_config, 'model_type', 'Unknown')}")
    print(f"  Vocab size: {getattr(text_config, 'vocab_size', 'Unknown')}")
    print(f"  Hidden size: {getattr(text_config, 'hidden_size', 'Unknown')}")

if hasattr(config, 'vision_config'):
    vision_config = config.vision_config
    print(f"  Vision model: {getattr(vision_config, 'model_type', 'Unknown')}")
    print(f"  Image size: {getattr(vision_config, 'image_size', 'Unknown')}")
    print(f"  Patch size: {getattr(vision_config, 'patch_size', 'Unknown')}")

# The revelation
print(f"\n💡 CONCLUSION:")
print(f"  SmolVLM appears to be BASED ON Idefics3 architecture!")
print(f"  This is why you're seeing Idefics3 classes everywhere.")
print(f"  SmolVLM is likely a fine-tuned or modified version of Idefics3.")

print("=" * 80)

🔍 MODEL ARCHITECTURE VERIFICATION

📋 Model Information:
  Model ID: HuggingFaceTB/SmolVLM-Instruct
  Model Class: Idefics3ForConditionalGeneration
  Model Module: transformers.models.idefics3.modeling_idefics3
  Processor Class: Idefics3Processor

⚙️  Model Configuration:
  Model Type: idefics3
  Architecture: ['Idefics3ForConditionalGeneration']
  Vision Config: Idefics3VisionConfig
  Text Config: LlamaConfig

🤔 Is this really SmolVLM?
  Config says model_type: idefics3
  Class name suggests: Idefics3

📊 Model Size:
  Total parameters: 2,246,272,880
  Trainable parameters: 2,246,272,880
  Model size: ~2.2B parameters

🔧 Key Config Details:
  Text model: llama
  Vocab size: 49155
  Hidden size: 2048
  Vision model: idefics3
  Image size: 384
  Patch size: 14

💡 CONCLUSION:
  SmolVLM appears to be BASED ON Idefics3 architecture!
  This is why you're seeing Idefics3 classes everywhere.
  SmolVLM is likely a fine-tuned or modified version of Idefics3.


In [13]:
# 🔍 CHECK FOR DEDICATED SMOLVLM CLASSES
print("=" * 80)
print("🔍 SEARCHING FOR DEDICATED SMOLVLM CLASSES")
print("=" * 80)

import transformers
print(f"\n📦 Transformers version: {transformers.__version__}")

# Check if SmolVLM classes exist
print(f"\n🔍 Looking for SmolVLM classes...")

try:
    from transformers import SmolVLMForConditionalGeneration
    print("✅ SmolVLMForConditionalGeneration - FOUND!")
except ImportError:
    print("❌ SmolVLMForConditionalGeneration - NOT FOUND")

try:
    from transformers import SmolVLMProcessor
    print("✅ SmolVLMProcessor - FOUND!")
except ImportError:
    print("❌ SmolVLMProcessor - NOT FOUND")

try:
    from transformers import SmolVLMImageProcessor  
    print("✅ SmolVLMImageProcessor - FOUND!")
except ImportError:
    print("❌ SmolVLMImageProcessor - NOT FOUND")

# Check what's available in transformers.models
print(f"\n📋 Available model types in transformers:")
import transformers.models
model_modules = [name for name in dir(transformers.models) if not name.startswith('_')]
smol_modules = [name for name in model_modules if 'smol' in name.lower()]
print(f"  SmolVLM-related modules: {smol_modules}")

# Check model mapping
print(f"\n🗺️  Model mapping for SmolVLM:")
try:
    from transformers import MODEL_FOR_VISION_2_SEQ_MAPPING, AutoConfig
    
    # Try to get config for SmolVLM
    config = AutoConfig.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
    print(f"  Config type: {type(config).__name__}")
    print(f"  Model type from config: {getattr(config, 'model_type', 'Unknown')}")
    
    # Check what model class this maps to
    if hasattr(config, 'model_type'):
        model_type = config.model_type
        if model_type in MODEL_FOR_VISION_2_SEQ_MAPPING:
            mapped_class = MODEL_FOR_VISION_2_SEQ_MAPPING[model_type]
            print(f"  Maps to class: {mapped_class.__name__}")
        else:
            print(f"  Model type '{model_type}' not in vision mapping")
            
except Exception as e:
    print(f"  Error checking mapping: {e}")

print(f"\n💡 RECOMMENDATION:")
if smol_modules:
    print(f"  Use dedicated SmolVLM classes if available!")
    print(f"  Try: from transformers import SmolVLMForConditionalGeneration, SmolVLMProcessor")
else:
    print(f"  No dedicated SmolVLM classes found.")
    print(f"  SmolVLM likely uses Idefics3 architecture as base.")
    print(f"  Consider updating transformers: pip install --upgrade transformers")

print("=" * 80)

🔍 SEARCHING FOR DEDICATED SMOLVLM CLASSES

📦 Transformers version: 4.53.1

🔍 Looking for SmolVLM classes...
✅ SmolVLMForConditionalGeneration - FOUND!
✅ SmolVLMProcessor - FOUND!
✅ SmolVLMImageProcessor - FOUND!

📋 Available model types in transformers:
  SmolVLM-related modules: ['SmolLM3Config', 'SmolLM3ForCausalLM', 'SmolLM3ForQuestionAnswering', 'SmolLM3ForSequenceClassification', 'SmolLM3ForTokenClassification', 'SmolLM3Model', 'SmolLM3PreTrainedModel', 'SmolVLMConfig', 'SmolVLMForConditionalGeneration', 'SmolVLMImageProcessor', 'SmolVLMImageProcessorFast', 'SmolVLMModel', 'SmolVLMPreTrainedModel', 'SmolVLMProcessor', 'SmolVLMVideoProcessor', 'SmolVLMVisionConfig', 'SmolVLMVisionTransformer', 'smollm3', 'smollm3.configuration_smollm3', 'smollm3.modeling_smollm3', 'smolvlm', 'smolvlm.configuration_smolvlm', 'smolvlm.image_processing_smolvlm', 'smolvlm.image_processing_smolvlm_fast', 'smolvlm.modeling_smolvlm', 'smolvlm.processing_smolvlm', 'smolvlm.video_processing_smolvlm']

🗺️  M

In [17]:
# 🧪 TEST LOADING WITH DEDICATED SMOLVLM CLASSES
print("=" * 80)
print("🧪 TESTING DEDICATED SMOLVLM CLASSES")
print("=" * 80)

# Try to load with dedicated SmolVLM classes
try:
    print("🔄 Attempting to load with SmolVLMForConditionalGeneration...")
    from transformers import SmolVLMForConditionalGeneration, SmolVLMProcessor
    
    # Load with dedicated classes
    smol_processor = SmolVLMProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
    smol_model = SmolVLMForConditionalGeneration.from_pretrained(
        "HuggingFaceTB/SmolVLM-Instruct",
        torch_dtype=torch.bfloat16,
        device_map="cuda",
    )
    
    print("✅ SUCCESS! Loaded with dedicated SmolVLM classes!")
    print(f"  Model type: {type(smol_model).__name__}")
    print(f"  Processor type: {type(smol_processor).__name__}")
    print(f"  Model module: {type(smol_model).__module__}")
    
    # Compare with current model
    print(f"\n🔍 Comparison with current model:")
    print(f"  Current model: {type(model).__name__}")
    print(f"  New model: {type(smol_model).__name__}")
    print(f"  Same class? {type(model) == type(smol_model)}")
    
    # Check if preprocessing differs
    print(f"\n🔍 Processor comparison:")
    print(f"  Current processor: {type(processor).__name__}")
    print(f"  New processor: {type(smol_processor).__name__}")
    print(f"  Same class? {type(processor) == type(smol_processor)}")
    
    # Update globals if different
    if type(model) != type(smol_model):
        print(f"\n🔄 Updating to use dedicated SmolVLM classes...")
        globals()['model'] = smol_model
        globals()['processor'] = smol_processor
        print("✅ Updated! Now using dedicated SmolVLM classes.")
    else:
        print(f"\n💡 Same classes - no difference between Auto and dedicated imports.")
        
except ImportError as e:
    print(f"❌ SmolVLM classes not available: {e}")
    print("💡 This confirms SmolVLM uses Idefics3 architecture")
    
except Exception as e:
    print(f"❌ Error loading with SmolVLM classes: {e}")

print("=" * 80)

🧪 TESTING DEDICATED SMOLVLM CLASSES
🔄 Attempting to load with SmolVLMForConditionalGeneration...


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


❌ SmolVLM classes not available: Package `num2words` is required to run SmolVLM processor. Install it with `pip install num2words`.
💡 This confirms SmolVLM uses Idefics3 architecture


In [18]:
# 🔍 CHECK FOR CAUSAL MASKING IN SMOLVLM
print("=" * 80)
print("🔍 ANALYZING CAUSAL MASKING IN SMOLVLM")
print("=" * 80)

# Check model configuration for causal attention
config = model.config
print(f"\n⚙️  Model Configuration Analysis:")
print(f"  Model type: {getattr(config, 'model_type', 'Unknown')}")

# Check text model configuration (where causal masking would be)
if hasattr(config, 'text_config'):
    text_config = config.text_config
    print(f"\n📝 Text Model Configuration:")
    print(f"  Text model type: {getattr(text_config, 'model_type', 'Unknown')}")
    print(f"  Is decoder: {getattr(text_config, 'is_decoder', 'Unknown')}")
    print(f"  Is encoder decoder: {getattr(text_config, 'is_encoder_decoder', 'Unknown')}")
    print(f"  Add cross attention: {getattr(text_config, 'add_cross_attention', 'Unknown')}")
    
    # Check for causal attention settings
    causal_attrs = ['is_causal', 'causal', 'use_causal_mask', 'causal_attention']
    for attr in causal_attrs:
        if hasattr(text_config, attr):
            print(f"  {attr}: {getattr(text_config, attr)}")

# Inspect the actual model layers for attention patterns
print(f"\n🔍 Model Architecture Analysis:")
print(f"  Model class: {type(model).__name__}")

# Look at the text model specifically
if hasattr(model, 'text_model'):
    text_model = model.text_model
    print(f"  Text model class: {type(text_model).__name__}")
    
    # Check if it has layers with attention
    if hasattr(text_model, 'layers'):
        print(f"  Number of layers: {len(text_model.layers)}")
        
        # Inspect first layer's attention
        if len(text_model.layers) > 0:
            first_layer = text_model.layers[0]
            print(f"  First layer class: {type(first_layer).__name__}")
            
            if hasattr(first_layer, 'self_attn'):
                attn = first_layer.self_attn
                print(f"  Attention class: {type(attn).__name__}")
                
                # Check for causal mask attributes
                causal_attrs = ['is_causal', 'causal', 'use_causal_mask']
                for attr in causal_attrs:
                    if hasattr(attn, attr):
                        print(f"  Attention {attr}: {getattr(attn, attr)}")

# Check what type of model this is based on architecture
print(f"\n🏗️  Architecture Analysis:")
model_name = type(model).__name__.lower()
if 'llama' in model_name or 'qwen' in model_name or 'phi' in model_name:
    print("  ✅ This appears to be based on a decoder-only architecture (likely causal)")
elif 'bert' in model_name or 'encoder' in model_name:
    print("  ❌ This appears to be based on an encoder architecture (likely not causal)")
elif 'idefics' in model_name or 'vlm' in model_name:
    print("  🤔 This is a vision-language model - causal masking depends on implementation")

# Test with a simple generation to see behavior
print(f"\n🧪 Testing Generation Behavior:")
test_prompt = "The quick brown"
test_inputs = processor(text=test_prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    # Generate with temperature=0 for deterministic output
    outputs = model.generate(
        **test_inputs,
        max_new_tokens=5,
        do_sample=False,
        pad_token_id=processor.tokenizer.eos_token_id
    )
    
generated_text = processor.decode(outputs[0], skip_special_tokens=True)
print(f"  Input: '{test_prompt}'")
print(f"  Generated: '{generated_text}'")
print(f"  → {'Looks like causal generation!' if len(generated_text) > len(test_prompt) else 'No new tokens generated'}")

print(f"\n💡 CONCLUSION:")
print(f"  SmolVLM appears to be a vision-language model that:")
print(f"  - Uses causal attention for text generation (like GPT)")
print(f"  - Processes images with bidirectional attention")
print(f"  - Combines both modalities for conditional generation")
print(f"  - Likely implements causal masking in the text decoder part")

print("=" * 80)

🔍 ANALYZING CAUSAL MASKING IN SMOLVLM

⚙️  Model Configuration Analysis:
  Model type: idefics3

📝 Text Model Configuration:
  Text model type: llama
  Is decoder: False
  Is encoder decoder: False
  Add cross attention: False

🔍 Model Architecture Analysis:
  Model class: Idefics3ForConditionalGeneration

🏗️  Architecture Analysis:
  🤔 This is a vision-language model - causal masking depends on implementation

🧪 Testing Generation Behavior:
  Input: 'The quick brown'
  Generated: 'The quick brown fox jumped over the lazy'
  → Looks like causal generation!

💡 CONCLUSION:
  SmolVLM appears to be a vision-language model that:
  - Uses causal attention for text generation (like GPT)
  - Processes images with bidirectional attention
  - Combines both modalities for conditional generation
  - Likely implements causal masking in the text decoder part


In [19]:
# 🔍 ANALYZING RUST vs PYTHON IMPLEMENTATION DIFFERENCES
print("=" * 80)
print("🔍 DEBUGGING RUST vs PYTHON PARITY ISSUES")
print("=" * 80)

# 1. Check current Python settings for maximum determinism
print("🐍 PYTHON IMPLEMENTATION ANALYSIS:")
print("=" * 50)

# Check model precision and settings
print(f"\n📊 Model Precision:")
print(f"  Model dtype: {model.dtype}")
print(f"  Model device: {next(model.parameters()).device}")

# Check generation config
gen_config = model.generation_config
print(f"\n⚙️  Generation Configuration:")
print(f"  do_sample: {getattr(gen_config, 'do_sample', 'Not set')}")
print(f"  temperature: {getattr(gen_config, 'temperature', 'Not set')}")
print(f"  top_p: {getattr(gen_config, 'top_p', 'Not set')}")
print(f"  top_k: {getattr(gen_config, 'top_k', 'Not set')}")
print(f"  repetition_penalty: {getattr(gen_config, 'repetition_penalty', 'Not set')}")
print(f"  length_penalty: {getattr(gen_config, 'length_penalty', 'Not set')}")
print(f"  no_repeat_ngram_size: {getattr(gen_config, 'no_repeat_ngram_size', 'Not set')}")

# Test with most deterministic settings
print(f"\n🎯 TESTING MAXIMUM DETERMINISM:")
print("=" * 40)

# Set all random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
import random
import numpy as np
random.seed(42)
np.random.seed(42)

# Test with ultra-deterministic generation
test_messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What color is the sky?"}
        ]
    },
]

test_prompt = processor.apply_chat_template(test_messages, add_generation_prompt=True)
test_inputs = processor(text=test_prompt, return_tensors="pt").to("cuda")

print(f"Input prompt: '{test_prompt}'")
print(f"Input token IDs: {test_inputs['input_ids'].tolist()}")

# Generate with maximum determinism
with torch.no_grad():
    outputs = model.generate(
        **test_inputs,
        max_new_tokens=10,
        do_sample=False,              # Greedy decoding
        temperature=1.0,              # Not used with do_sample=False, but set anyway
        top_p=1.0,                    # Not used with do_sample=False
        top_k=0,                      # Not used with do_sample=False
        repetition_penalty=1.0,       # No repetition penalty
        length_penalty=1.0,           # No length penalty
        no_repeat_ngram_size=0,       # No n-gram blocking
        pad_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,               # Use KV cache for consistency
    )

generated_text = processor.decode(outputs[0], skip_special_tokens=True)
output_tokens = outputs[0].tolist()

print(f"\nGenerated tokens: {output_tokens}")
print(f"Generated text: '{generated_text}'")

# Extract just the new tokens
input_length = test_inputs['input_ids'].shape[1]
new_tokens = output_tokens[input_length:]
new_text = processor.tokenizer.decode(new_tokens, skip_special_tokens=True)

print(f"\nNew tokens only: {new_tokens}")
print(f"New text only: '{new_text}'")

print(f"\n🔧 POTENTIAL SOURCES OF DIFFERENCES:")
print("=" * 50)
print("""
1. 🔢 NUMERICAL PRECISION:
   - Python: Uses PyTorch's bf16/f32 precision
   - Rust: Candle's precision might differ slightly
   - Solution: Ensure exact same dtype operations

2. 🧮 ATTENTION COMPUTATION:
   - Softmax numerical stability differences
   - Matrix multiplication order/accumulation
   - Causal mask application timing
   - Solution: Match exact computation order

3. 🎭 LAYER NORMALIZATION:
   - Epsilon values (1e-5 vs 1e-6)
   - Computation order (x-mean)/std vs x/std - mean/std
   - Solution: Use exact same epsilon and formula

4. 🔄 TOKENIZATION:
   - Preprocessing differences
   - Special token handling
   - Solution: Use identical tokenizer and special tokens

5. 🖼️  IMAGE PREPROCESSING:
   - Resize/normalization differences
   - Padding strategies
   - Patch splitting logic
   - Solution: Match exact preprocessing pipeline

6. 💾 WEIGHT LOADING:
   - Tensor layout differences (row vs column major)
   - Weight precision during loading
   - Solution: Verify weight values match exactly

7. 🎲 GENERATION LOGIC:
   - Logit processing order
   - Temperature application
   - Token selection tie-breaking
   - Solution: Match exact generation pipeline
""")

print(f"\n✅ ACHIEVING EXACT PARITY:")
print("=" * 30)
print("""
To get exact parity:

1. 🔍 INTERMEDIATE OUTPUT COMPARISON:
   - Print logits before softmax in both implementations
   - Compare attention weights layer by layer
   - Verify embeddings and hidden states match

2. 🧪 UNIT TESTING:
   - Test individual components (attention, MLP, etc.)
   - Use simple inputs (zeros, ones, known patterns)
   - Compare outputs at each layer

3. 🎯 WEIGHT VERIFICATION:
   - Extract weights from Python model
   - Verify they load identically in Rust
   - Check for any transpose/reshape differences

4. 📊 PRECISION CONTROL:
   - Use identical dtypes throughout pipeline
   - Implement identical numerical algorithms
   - Handle edge cases (div by zero, etc.) the same way
""")

print("=" * 80)

The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 DEBUGGING RUST vs PYTHON PARITY ISSUES
🐍 PYTHON IMPLEMENTATION ANALYSIS:

📊 Model Precision:
  Model dtype: torch.bfloat16
  Model device: cuda:0

⚙️  Generation Configuration:
  do_sample: False
  temperature: 1.0
  top_p: 1.0
  top_k: 50
  repetition_penalty: 1.0
  length_penalty: 1.0
  no_repeat_ngram_size: 0

🎯 TESTING MAXIMUM DETERMINISM:
Input prompt: '<|im_start|>User: What color is the sky?<end_of_utterance>
Assistant:'
Input token IDs: [[1, 11126, 42, 1812, 2380, 314, 260, 6376, 47, 49154, 198, 9519, 9531, 42]]

Generated tokens: [1, 11126, 42, 1812, 2380, 314, 260, 6376, 47, 49154, 198, 9519, 9531, 42, 10062, 30, 49154]
Generated text: 'User: What color is the sky?
Assistant: Blue.'

New tokens only: [10062, 30, 49154]
New text only: ' Blue.'

🔧 POTENTIAL SOURCES OF DIFFERENCES:

1. 🔢 NUMERICAL PRECISION:
   - Python: Uses PyTorch's bf16/f32 precision
   - Rust: Candle's precision might differ slightly
   - Solution: Ensure exact same dtype operations

2. 🧮 ATTENTION COMPUT

In [20]:
# 🔧 PRACTICAL DEBUGGING STEPS
print("=" * 80)
print("🔧 PRACTICAL DEBUGGING FOR RUST PARITY")
print("=" * 80)

print("""
📋 STEP-BY-STEP DEBUGGING CHECKLIST:

1. 🎯 FIRST: Test with simple text-only input
   - Remove image processing from the equation
   - Use: "Hello world" → check if outputs match
   - This isolates text generation differences

2. 🔍 INTERMEDIATE OUTPUTS:
   Add debug prints in your Rust code at these points:
   
   a) After tokenization:
      println!("Input tokens: {:?}", input_ids);
   
   b) After embeddings:
      println!("Embeddings mean: {}", embeddings.mean());
   
   c) After each transformer layer:
      println!("Layer {} output mean: {}", i, hidden_states.mean());
   
   d) Before final logits:
      println!("Pre-logits mean: {}", pre_logits.mean());
   
   e) Final logits for next token:
      println!("Logits for position {}: {:?}", pos, logits.slice());

3. 🎲 GENERATION COMPARISON:
   Run both implementations with identical settings:
   
   Python:
   ```python
   outputs = model.generate(
       input_ids=torch.tensor([[1, 2, 3]]),
       max_new_tokens=1,
       do_sample=False,
       use_cache=False,  # Disable KV cache for debugging
   )
   ```
   
   Rust:
   ```rust
   let outputs = model.generate(
       &input_ids,
       1, // max_new_tokens
       false, // do_sample
   );
   ```

4. 🔢 NUMERICAL DEBUGGING:
   Check these specific values match:
   
   - Embedding weights: model.embed_tokens.weight[0, :10]
   - Layer norm epsilon: config.layer_norm_eps  
   - Attention head dimensions
   - RMSNorm vs LayerNorm implementation
""")

# Let's extract some key numerical values for comparison
print(f"\n📊 KEY NUMERICAL VALUES FOR RUST COMPARISON:")
print("=" * 50)

# Get some embeddings to compare
if hasattr(model, 'get_input_embeddings'):
    embeddings = model.get_input_embeddings()
    sample_embeddings = embeddings.weight[:5, :5]  # First 5 tokens, first 5 dims
    print(f"\nEmbedding weights sample (first 5x5):")
    print(sample_embeddings)

# Get model config values
config = model.config
if hasattr(config, 'text_config'):
    text_config = config.text_config
    print(f"\nKey config values:")
    print(f"  hidden_size: {getattr(text_config, 'hidden_size', 'N/A')}")
    print(f"  num_attention_heads: {getattr(text_config, 'num_attention_heads', 'N/A')}")
    print(f"  num_key_value_heads: {getattr(text_config, 'num_key_value_heads', 'N/A')}")
    print(f"  intermediate_size: {getattr(text_config, 'intermediate_size', 'N/A')}")
    print(f"  rms_norm_eps: {getattr(text_config, 'rms_norm_eps', 'N/A')}")
    print(f"  rope_theta: {getattr(text_config, 'rope_theta', 'N/A')}")
    print(f"  vocab_size: {getattr(text_config, 'vocab_size', 'N/A')}")

# Test a minimal case
print(f"\n🧪 MINIMAL TEST CASE:")
print("=" * 30)

# Single token input
single_token = torch.tensor([[1]]).to("cuda")  # Usually BOS token
print(f"Single token input: {single_token}")

with torch.no_grad():
    # Get logits for next token
    outputs = model(input_ids=single_token)
    logits = outputs.logits[0, -1, :]  # Last position logits
    
    # Get top 5 most likely next tokens
    top_logits, top_indices = torch.topk(logits, 5)
    
    print(f"Top 5 next token logits: {top_logits.tolist()}")
    print(f"Top 5 next token IDs: {top_indices.tolist()}")
    
    # Convert to tokens
    top_tokens = [processor.tokenizer.decode([idx]) for idx in top_indices]
    print(f"Top 5 next tokens: {top_tokens}")

print(f"\n💡 DOES IT MATTER?")
print("=" * 20)
print("""
Small differences might be acceptable if:
✅ Generated text is semantically similar
✅ Differences are in low-probability tokens
✅ Overall model performance is maintained

Exact parity is needed if:
❌ Building production systems requiring reproducibility
❌ Comparing against benchmarks
❌ Debugging model behavior
❌ Academic research requiring exact reproduction

For most applications, "close enough" is often sufficient!
""")

print("=" * 80)

🔧 PRACTICAL DEBUGGING FOR RUST PARITY

📋 STEP-BY-STEP DEBUGGING CHECKLIST:

1. 🎯 FIRST: Test with simple text-only input
   - Remove image processing from the equation
   - Use: "Hello world" → check if outputs match
   - This isolates text generation differences

2. 🔍 INTERMEDIATE OUTPUTS:
   Add debug prints in your Rust code at these points:

   a) After tokenization:
      println!("Input tokens: {:?}", input_ids);

   b) After embeddings:
      println!("Embeddings mean: {}", embeddings.mean());

   c) After each transformer layer:
      println!("Layer {} output mean: {}", i, hidden_states.mean());

   d) Before final logits:
      println!("Pre-logits mean: {}", pre_logits.mean());

   e) Final logits for next token:
      println!("Logits for position {}: {:?}", pos, logits.slice());

3. 🎲 GENERATION COMPARISON:
   Run both implementations with identical settings:

   Python:
   ```python
   outputs = model.generate(
       input_ids=torch.tensor([[1, 2, 3]]),
       max_new_to